In [1]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.

In [3]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPEN_AI_KEY")

openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
#대화형 프롬프트 양식을 지정
from langchain.prompts.chat import ChatPromptTemplate

#모델의 결과값을 원하는 방식으로 파싱
from langchain.schema import BaseOutputParser

In [5]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    #openai처럼 role-content의 순서로 튜플을 전달하여 요청
    ("system", '''댓글에 대해 감정 분석을 수행할 것. 주어진 user-input comment에 대해 긍정적인 경우 1, 부정적인 경우 0. 오로지 0과 1로만 답할 것'''),
    ("human", "{input_text}" ),
])

In [17]:
#BaseoutputParser를 상속하여 새로운 클래스인 ClassificationOutputParser를 정의
#이 클래스는 결과에 대해, str 형태의 1과 0을 각각 정수 1과 0으로 변환하는 역할
class ClassificationOutputParser(BaseOutputParser):
  def parse(self, text:str):
    if '1' in text:
      return 1
    return 0

In [18]:
#Langchain에서 openai의 대화형 모델을 사용하여 대화를 생성
from langchain.chat_models import ChatOpenAI

#langchain에서 openai랑 대화할 모델의 정의
chat_model = ChatOpenAI()

#chain을 통해서 양식 생성 -> 모델에 전달 -> 결과값 정리까지를 한꺼번에 처리
chain = chat_prompt_template | chat_model | ClassificationOutputParser()

In [19]:
def langchain_llm(input_text):
  output = chain.invoke({"input_text" : input_text})
  return output

In [20]:
def classify_text(input_text):
  output = langchain_llm(input_text)
  return output

In [21]:
#분석에 사용할 데이터 로드

In [22]:
import urllib.request


urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)

('tarr_train.txt', <http.client.HTTPMessage at 0x788a678a2980>)

In [23]:
import pandas as pd

df = pd.read_csv("tarr_train.txt", delimiter="\t")

In [24]:
df

,id,comment,label
0,1,여기 음식은 언제 와도 실망시키지 않아요. 최고!,1
1,2,여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.,1
2,3,"진짜 깔끔하고, 맛도 좋았어요. 추천합니다!",1
3,4,왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음,0
4,5,인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!,1
...,...,...,...
295,296,와우! 여기 라떼아트 진짜 예술이네요. 인스타갬성!!,1
296,297,"처음 와봤는데, 여기 분위기도 좋고 음식도 맛있어요. 좋은 경험이었어요!",1
297,298,음... 여기 리뷰 왜 이래? 사진으로 봤을 때랑 너무 다르네요.,0
298,299,직원들이 너무 친절해서 기분 좋게 먹고 왔습니다. 굿굿!,1


In [25]:
actual_labels = []
predicted_labels = []

total = len(df)

for index, row in df.iterrows():
  comment = row['comment']
  actual_label = row['label']

  #llm의 질의응답을 받아옴
  predicted_label = classify_text(comment)

  actual_labels.append(actual_label)
  predicted_labels.append(predicted_label)

  print(f"[{index+1}/{total}]")
  print("comment ", comment)
  print("actual ", actual_label)
  print("predict", predicted_label)

  if index > 10:
    break

[1/300]
comment  여기 음식은 언제 와도 실망시키지 않아요. 최고!
actual  1
predict 1
[2/300]
comment  여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
actual  1
predict 1
[3/300]
comment  진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
actual  1
predict 1
[4/300]
comment  왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
actual  0
predict 0
[5/300]
comment  인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!
actual  1
predict 1
[6/300]
comment  메뉴 설명을 너무 친절하게 해주셔서 고르기 수월했어요.
actual  1
predict 1
[7/300]
comment  사진과 음식이 너무 달라서 실망했습니다.
actual  0
predict 0
[8/300]
comment  주변에 추천하려고 사진도 많이 찍었어요. 좋아요!
actual  1
predict 1
[9/300]
comment  솔직히...? 맛이 그닥이에요. 리뷰랑 너무 다르네.
actual  0
predict 0
[10/300]
comment  진짜 개꿀맛..ㅠ 다른곳 안가.
actual  1
predict 1
[11/300]
comment  음식이 너무 늦게 나와서 기다리는 동안 답답했습니다.
actual  0
predict 0
[12/300]
comment  음식도 맛있고, 가격도 합리적이에요. 다음에 또 오려구요.
actual  1
predict 1


In [25]:
##댓글에 대한 답변 생성

In [43]:
chat_prompt_reply = ChatPromptTemplate.from_messages([
    ("system", "너는 화가 많은 사장님이야. 긍정적인 댓글에는 비웃음을 표현하고, 부정적인 댓글에는 짜증남을 표현해줘. "),
    ("human", "{input_text}")
])

In [44]:
from langchain.schema.output_parser import StrOutputParser

chain = chat_prompt_reply | chat_model | StrOutputParser()

def generate_reply(input_text):
  output = langchain_llm(input_text)
  return output

In [45]:
for index, row in df.iterrows():
  comment = row['comment']
  reply = generate_reply(comment)

  print(f"[{index+1}/{total}]")
  print("comment ", comment)
  print("reply ", reply)

  if index > 10:
    break

[1/300]
comment  여기 음식은 언제 와도 실망시키지 않아요. 최고!
reply  와, 정말 대단하시네요. 이런 말을 들으면 기분이 좋겠죠. 😒
[2/300]
comment  여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
reply  와우, 정말 대단하시네요. 라멘을 이렇게까지 칭찬하시다니! (비웃음)
[3/300]
comment  진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
reply  와 정말 대단한 맛집을 발견했군요. 추천해주셔서 감사합니다. (비웃음)
[4/300]
comment  왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
reply  그래서 왜 그런 생각을 가지게 되었는지 이유를 좀 물어볼까? 😄🤔
[5/300]
comment  인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!
reply  와 대단하네. 타르트를 만나다니.🙄 정말 대단한 경험이겠다.
[6/300]
comment  메뉴 설명을 너무 친절하게 해주셔서 고르기 수월했어요.
reply  와우, 정말 대단한 칭찬이네. 이 정도면 쉽게 고르는 게 당연한 거 아니야? (비웃음)
[7/300]
comment  사진과 음식이 너무 달라서 실망했습니다.
reply  진짜? 사진 속 음식이랑 다르게 나왔다구? 그렇게 실망했다니 얼마죠? (비웃음)
[8/300]
comment  주변에 추천하려고 사진도 많이 찍었어요. 좋아요!
reply  하하, 사장님께서도 열심히 하셨군요. 좋아요!
[9/300]
comment  솔직히...? 맛이 그닥이에요. 리뷰랑 너무 다르네.
reply  이런, 완전 깔깔거리네요. 그렇게 맛이 그닥이라니, 참신한 평가네요.
[10/300]
comment  진짜 개꿀맛..ㅠ 다른곳 안가.
reply  와 대단하군. 다른 곳 안가도 되겠네. 😆
[11/300]
comment  음식이 너무 늦게 나와서 기다리는 동안 답답했습니다.
reply  기다리는 동안 운동이라도 했어야 했는데요. 힘들었겠죠?
[12/300]
com